In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import time
import datetime

In [3]:
index_choice = 'sp500'

In [4]:
SMALL_SIZE = 6
MEDIUM_SIZE = 8
BIGGER_SIZE = 10

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)

In [5]:
file_path = os.path.join("temp",f"{index_choice}",f"{index_choice}_final_smoothed.csv")
final_df = pd.read_csv(file_path)
final_df["datetime"] = final_df["date"].apply(lambda s: datetime.datetime.strptime(s, "%Y-%m-%d"))

In [6]:
def normalize (df):
    min = df.min()
    max = df.max()
    x = df
    y = (x-min) / (max-min)
    return y

In [8]:
industry_choice = ["Airlines","Internet Retail", "Electronic Gaming & Multimedia"]

industry_extract = final_df[final_df["industry"].isin(industry_choice)]

industry_extract["normalized_open"]= industry_extract.groupby("ticker")["open_x"].apply(lambda x: normalize (x))

industry_extract_grouped = industry_extract.groupby(["datetime","industry"]).mean("normalized_open").reset_index()

In [ ]:
plt.scatter(normalized_open.Airlines, retscompnormalized_open.Internet Retail)

In [9]:
pd.plotting.scatter_matrix(industry_extract_grouped, diagonal='kde', figsize=(20, 20))

KeyboardInterrupt: 